# SALUD MURCIA

In [1]:
from pandas import read_csv
import pandas as pd
from pandas.plotting import lag_plot
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import io

Lectura del conjunto de datos obtenido de la pagina de salud de Murcia 
https://www.murciasalud.es/pagina.php?id=458869
Recogidos desde el 08/03/2020 hasta el dia 25/03/2020

In [2]:
lectura = read_csv('../Datos_originales/casos_diarios_total_regional.csv', skiprows = 4, encoding='latin-1', sep = ';', na_values = ['NaN', 'NaT'])

# Headers = columnas de mi conjunto de datos. Si no especificamos asume que son las de la primera fila leida
# Skiprows = n: con n = numero de filas que quiero saltar. Las cuatro primeras son texto -> las saltamos
# Encoding = 'latin-1': para que no de errores de codificacion de algunos caracteres 
# Sep = ';': delimitador que separa las columnas
# Index_col = 0: usar la primera columna (fechas) como filas del dataframe

Renombramos los nombres de algunas de las columnas para que queden mas compactos

In [3]:
datos = lectura.rename(columns={'Pruebas_PCR_Ag':'Pruebas',
                                'Casos_incidentes_PCR/Ag+':'Incidentes',
                                'Hospitalizados': 'H',
                                'Hospitaliz_UCI':'H_UCI',
                                'Asintomatico_hombre_<45años': 'Asint_H_<45',
                                'Asintomatico_hombre_45a64años': 'Asint_H_45a64',
                                'Asintomatico_hombre_+65años': 'Asint_H_+65',
                                'Sintomatico_hombre_<45años': 'Sint_H_<45',
                                'Sintomatico_hombre_45a64años': 'Sint_H_45a64',
                                'Sintomatico_hombre_+65años': 'Sint_H_+65',
                                'Asintomatico_mujer_<45años': 'Asint_M_<45',
                                'Asintomatico_mujer_45a64años': 'Asint_M_45a64',
                                'Asintomatico_mujer_+65años': 'Asint_M_+65',
                                'Sintomatico_mujer_<45años': 'Sint_M_<45',
                                'Sintomatico_mujer_45a64años': 'Sint_M_45a64',
                                'Sintomatico_mujer_+65años': 'Sint_M_+65'})

Convertimos los datos leidos en un dataFrame y lo mostramos para ver su apariencia

In [4]:
df_salud_murcia = pd.DataFrame(datos)
df_salud_murcia.head(3)

,Fecha,Pruebas,Incidentes,H,H_UCI,Defunciones,Asint_H_<45,Asint_H_45a64,Asint_H_+65,Sint_H_<45,Sint_H_45a64,Sint_H_+65,Asint_M_<45,Asint_M_45a64,Asint_M_+65,Sint_M_<45,Sint_M_45a64,Sint_M_+65
0,08/03/2020,130.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,09/03/2020,21.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,10/03/2020,39.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0


Lo siguiente que haremos es indexar el dataframe por la columna 'Fecha'. Para ello, primero debemos convertir la misma en un objeto de tipo DatatimeIndex. Usamos la funcion 'to_datetime' y le indicamos el formato en el que se encuentran las fechas para que se lea y procese bien. Posteriormente eliminamos la columna del dataFrame e insertamos el nuevo DatatimeIndex como íncide del dataframe. Finalmente mostramos el resultado.

In [5]:
format = '%d/%m/%Y'
datetime_fecha = pd.to_datetime(df_salud_murcia['Fecha'], format = format)
df_salud_murcia = df_salud_murcia.drop(['Fecha'], axis=1)
df_salud_murcia.insert(0,'Fecha', datetime_fecha)
df_salud_murcia = df_salud_murcia.set_index('Fecha')
df_salud_murcia.head(3)

,Pruebas,Incidentes,H,H_UCI,Defunciones,Asint_H_<45,Asint_H_45a64,Asint_H_+65,Sint_H_<45,Sint_H_45a64,Sint_H_+65,Asint_M_<45,Asint_M_45a64,Asint_M_+65,Sint_M_<45,Sint_M_45a64,Sint_M_+65
Fecha,,,,,,,,,,,,,,,,,
2020-03-08,130.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2020-03-09,21.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2020-03-10,39.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0


Con la siguiente función eliminamos todos aquellos valores que se almacenan o completan como NaN. 

In [6]:
df_salud_murcia.dropna(axis = 0, inplace = True)

## Sumando asintomáticos
En esta sección vamos a sumar a todos los asintomáticos (por fecha) y añadirlos en una nueva serie
del dataframe a la que llamaré "Asintomaticos"

In [7]:
df_salud_murcia['Asintomaticos'] = df_salud_murcia['Asint_H_<45'] + df_salud_murcia['Asint_H_45a64'] + df_salud_murcia['Asint_H_+65'] + df_salud_murcia['Asint_M_<45'] + df_salud_murcia['Asint_M_45a64'] + df_salud_murcia['Asint_M_+65']

A continuación lo que hacemos es guardarla en un archivo de extensión .csv 

In [8]:
df_salud_murcia.to_csv(path_or_buf = '../Datos_preprocesados./datos_murcia_mas_asintomaticos.csv')

## Movilidad INE

In [9]:
def parseador_fechas_marjun(nombre):
    # Cogemos el día
    dia = nombre[4:6]
    mes = nombre[6:9]
    anio = nombre[10:14]
    if mes == 'MAR':
        mes = '03'
    if mes == 'ABR':
        mes = '04'
    if mes == 'MAY':
        mes = '05'
    if mes == 'JUN':
        mes = '06'
    return(anio+'-'+dia+'-'+mes)

In [10]:
def parseador_fechas_jundic(nombre):
    # Cogemos la longitud total y nos quedamos con las fechas
    l = len(nombre)
    f = nombre[l-9:l-5]
    # Obtenemos el dia y el mes
    dia = f[0:2]
    mes = f[2:4]
    # El añi es siempre 2020
    anio = '2020'
    # Devolvemos la fecha en su formato
    return(anio+'-'+dia+'-'+mes)

Al igual que hicimos con el dataFrame de los datos de salud que leímos al principio, debemos transformar las fechas en objetos de tipo DatatimeIndex. Posteriormente las ordenaremos para que se muestren en orden cronológico. Para ello usamos la función 'to_datetime', que convierte nuestra lista de fechas en DatatimeIndex. Acto seguido creamos el dataframe con las entradas, las salidas a Murcia y las fechas y las ponemos como índice del dataframe. Ahora solo queda ordenarlo. Añadimos tres columnas adicionales al dataframe que son los meses, dias y años de las fechas. Ordenamos según estos valores y posteriormente las eliminamos del dataframe. Mostramos el resultado.

In [11]:
def conversor_a_dataframe(lista_fechas, lista_entradas, lista_salidas):
    format = '%Y-%d-%m'
    # Convertimos en DatatimeIndex
    fechas_serie = pd.to_datetime(lista_fechas, format = format)
    # Creamos dataframe con las entradas salidas y fechas
    df_entradas_salidas = pd.DataFrame(list(zip(lista_entradas, lista_salidas)), columns = ['Entradas', 'Salidas'], index = fechas_serie).rename_axis('Fecha', axis = 0)
    # Definimos las series para ordenar
    df_entradas_salidas['month'] = fechas_serie.month
    df_entradas_salidas['year'] = fechas_serie.year
    df_entradas_salidas['day'] = fechas_serie.day
    # Ordenamos por mes-dia-año
    df_entradas_salidas = df_entradas_salidas.sort_values(['month','day','year']).drop(columns=['day', 'month', 'year'])
    return df_entradas_salidas

### Movilidad mar-jun

In [12]:
ruta = '../Datos_originales/mar-jun/'

lista_entradas = []
lista_salidas = []
lista_fechas = []

# Cogemos los archivos .zip diarios
archivos = os.listdir(ruta)

# Iteramos sobre cada uno de estos
for a in archivos:
    # Los leemos con zipfile
    z = zipfile.ZipFile(ruta+a, 'r')
    # Iteramos en cada uno de los ficheros del zip
    for az in z.namelist():
        # Cogemos los .csv que nos interesan
        if "PobxCeldasOrigen" in az:
            # Nos quedamos con el dia (será el mismo para entrada y salida)
            fecha = parseador_fechas_marjun(az)
            lista_fechas.append(fecha)
            # Como son archivos .csv leemos
            df_f = pd.read_csv(z.open(az), sep=';', encoding = 'latin-1')
            # Obtenemos solamente las salidas de Murcia
            df_murcia = df_f[df_f['NOMBRE_CELDA_ORIGEN '].str.contains("Murcia \(")]
            # Sumamos toda la población registrada que sale de Murcia
            suma_salidas_murcia = df_murcia['POB_SALE '].sum()
            # Añadimos este dato a la lista de salidas
            lista_salidas.append(suma_salidas_murcia)
            
        if "PobxCeldasDestino" in az:
            # Como son archivos .csv leemos
            df_f = pd.read_csv(z.open(az), sep=';', encoding = 'latin-1')
            # Obtenemos solamente las entradas a Murcia
            df_murcia = df_f[df_f['NOMBRE_CELDA_DESTINO '].str.contains("Murcia \(")]
            # Sumamos toda la población registrada que llega a Murcia
            suma_entradas_murcia = df_murcia['POB_LLEGA '].sum()
            # Añadimos este dato a la lista de salidas
            lista_entradas.append(suma_entradas_murcia)

### Movilidad jun-dic

In [13]:
def procesamiento_tablas(nombre,z):
    # Obtenemos la fecha
    fecha = parseador_fechas_jundic(nombre)
    lista_fechas.append(fecha)
    # Leemos el fichero excel
    df = pd.read_excel(z.open(nombre))
    # Cogemos sólamente las áreas de Murcia ciudad
    df_murcia = df[df['Nombre del Área'].str.contains("Murcia \(")]
    # Entradas
    suma_entradas_murcia = df_murcia['Población residente encontrada durante el día en otra área (C)'].sum()
    lista_entradas.append(suma_entradas_murcia)
    # Salidas
    suma_salidas_murcia = df_murcia['Población no residente que se localiza durante el día en esta área (D)'].sum()
    lista_salidas.append(suma_salidas_murcia)

In [14]:
ruta = '../Datos_originales/jun-dic/'

# Cogemos los archivos .zip mensuales
archivos = os.listdir(ruta)

for a in archivos:
    with zipfile.ZipFile(ruta+a) as z:
        for f in z.namelist():
            #print(f)
            # Todos excepto noviembre y diciembre tienen directamente las tablas
            if "Tabla 1.2" in f:
                procesamiento_tablas(f,z)
            # Extraemos los .zip diarios de noviembre y diciembre
            if ".zip" in f and ("Noviembre" in f or "Diciembre" in f):
                with z.open(f) as d:
                    with zipfile.ZipFile(d) as z2:
                        for h in z2.namelist():
                            if "Tabla 1.2" in h:
                                procesamiento_tablas(h,z2)
            

## DataFrame entradas y salidas Murcia

In [15]:
df_entradas_salidas_murcia = conversor_a_dataframe(lista_fechas, lista_entradas, lista_salidas)

In [17]:
df_entradas_salidas_murcia.to_csv(path_or_buf = '../Datos_preprocesados./datos_entradas_salidas_murcia_ine.csv')

## DataFrame salud y entradas y salidas Murcia

In [16]:
df_final_ine = pd.concat([df_salud_murcia, df_entradas_salidas_murcia], axis=1)

In [18]:
df_final_ine.to_csv(path_or_buf = '../Datos_preprocesados./datos_murcia_mas_entradas_salidas_ine.csv')

## Visualización de todas las series (bucle)

A continuación se realiza un bucle en el que se visualizan los datos de todas las series del dataframe

In [ ]:
num = 0

# Initialize the figure style
plt.style.use('seaborn-white')
 
# create a color palette

palette = plt.get_cmap('prism')

for column in df_final_ine:
    
    num = num + 1
 
    # Find the right spot on the plot
    plt.subplot(5,4, num)
 
    if "Salidas" in column or "Entradas" in column:
        serie = df_final_ine[column]
        serie.plot(style = '.', color = palette(num), linewidth = 0.8, alpha = 0.9, label = column)
    else:
        # Plot the lineplot
        plt.plot(df_final_ine[column], marker = '', color = palette(num), linewidth = 0.8, alpha = 0.9, label = column)

    # Add title
    plt.title(column, loc ='center', fontsize = 12, fontweight = 0, color = palette(num))

plt.subplots_adjust(left = 0.01, right = 4, top = 5, bottom = 0.01, hspace = 0.2, wspace = 0.2)


# Show the graph
plt.show()

Las gráficas tienen mucha similitud entre ellas, lo que es bastante lógico.
El aumento/disminución del número de casos positivos se ve directamente relacionado con 
el aumento de hospitalizaciones, defunciones, asintomáticos, sintomáticos,...

## Añadiendo movilidad MITMA

El código del municipio de Murcia es 30030. Usaremos este código para acceder a los datos de movilidad de la ciudad
del ministerio recogidos en la carpeta mitma_movilidad. Para cada mes tenemos un archivo .tar con ficheros .txt
correspondientes a cada dia

In [19]:
def parseador_fechas_mitma(nombre):
    # Cogemos el día
    dia = nombre[6:8]
    mes = nombre[4:6]
    anio = nombre[0:4]
    return(anio+'-'+dia+'-'+mes)

In [20]:
ruta = '../Datos_originales/mitma_movilidad/202002_maestra1_mitma_municipio'

lista_fechas = []
lista_entradas = []
lista_salidas = []

# Cogemos los archivos .tar mensuales
archivos = os.listdir(ruta)

for a in archivos:
    if "._" not in a:
        # Obtenemos la fecha
        fecha = parseador_fechas_mitma(a)
        lista_fechas.append(fecha)
        # Leemos el archivo correspondiente
        df = pd.read_csv(ruta+'/'+a, compression='gzip', sep='|', header=0)
        # Nos quedamos solo con Murcia código 30030
        df_s = df[df['origen'].str.contains('30030', na=False)]
        df_e = df[df['destino'].str.contains('30030', na=False)]
        # Sumamos todos los viajes que son las entradas y salidas
        salidas = df_s['viajes'].sum()
        entradas = df_e['viajes'].sum()
        # Añadimos a las listas
        lista_salidas.append(salidas)
        lista_entradas.append(entradas)

In [21]:
df_e.head(5)

,fecha,origen,destino,periodo,distancia,viajes,viajes_km
8645,20200229,02003,30030,2,100+,5.827,724.622
8646,20200229,02003,30030,4,100+,5.087,617.715
8647,20200229,02003,30030,5,100+,16.623,2361.378
8648,20200229,02003,30030,6,100+,7.399,931.785
8649,20200229,02003,30030,7,100+,31.482,4058.223


## Empezando con predictores

Para trabajar con las distintas series temporales y realizar machine learning vamos en primer lugar a sumar todos
los sintomáticos y añadirlos a una nueva serie del dataframe al igual que hicimos con los asintomáticos. Posteriormente
vamos a crear un nuevo dataframe donde eliminemos las series de sintomaticos y asintomaticos por edades, quedándonos
solamente con las nuevas series

In [22]:
df_final_ine['Sintomaticos'] = df_salud_murcia['Sint_H_<45'] + df_salud_murcia['Sint_H_45a64'] + df_salud_murcia['Sint_H_+65'] + df_salud_murcia['Sint_M_<45'] + df_salud_murcia['Sint_M_45a64'] + df_salud_murcia['Sint_M_+65']
df_final_ine.head(2)

,Pruebas,Incidentes,H,H_UCI,Defunciones,Asint_H_<45,Asint_H_45a64,Asint_H_+65,Sint_H_<45,Sint_H_45a64,...,Asint_M_<45,Asint_M_45a64,Asint_M_+65,Sint_M_<45,Sint_M_45a64,Sint_M_+65,Asintomaticos,Entradas,Salidas,Sintomaticos
Fecha,,,,,,,,,,,,,,,,,,,,,
2020-03-08,130.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,NaN,NaN,1.0
2020-03-09,21.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,NaN,NaN,4.0


In [23]:
df_final_ine = df_final_ine.drop(['Asint_H_<45', 'Asint_H_45a64', 'Asint_H_+65', 'Asint_M_<45', 'Asint_M_45a64', 'Asint_M_+65', 'Sint_H_<45', 'Sint_H_45a64', 'Sint_H_+65', 'Sint_M_<45', 'Sint_M_45a64', 'Sint_M_+65'], axis=1)
df_final_ine.head(3)

,Pruebas,Incidentes,H,H_UCI,Defunciones,Asintomaticos,Entradas,Salidas,Sintomaticos
Fecha,,,,,,,,,
2020-03-08,130.0,1.0,0.0,0.0,0.0,2.0,NaN,NaN,1.0
2020-03-09,21.0,3.0,0.0,0.0,0.0,0.0,NaN,NaN,4.0
2020-03-10,39.0,6.0,0.0,0.0,0.0,0.0,NaN,NaN,4.0


In [24]:
df_final_ine.to_csv(path_or_buf = '../Datos_preprocesados./datos_ine_final.csv')

Usamos el método de la ventana deslizante. Pretendemos predecir el número de casos en el día 't' a partir de los
tres días anteriores 't-3', 't-2', 't-1'

In [26]:
df_predictor = df_final_ine.copy()
columnas = df_predictor.columns
columnas = columnas.drop("Incidentes")
lista = []
# Añadimos lag a todas las series excepto a Incidentes, que es la que queremos predecir
for c in columnas:
    datos = pd.DataFrame(df_predictor[c].values, index = df_predictor.index)
    for i in [3,2,1]:
        df_predictor.insert(len(df_predictor.columns), c+' t-'+str(i),datos.shift(i))
        lista.append(df_predictor.columns)

df_predictor = df_predictor.drop(columnas, axis=1)

In [27]:
df_predictor.head(10)

,Incidentes,Pruebas t-3,Pruebas t-2,Pruebas t-1,H t-3,H t-2,H t-1,H_UCI t-3,H_UCI t-2,H_UCI t-1,...,Asintomaticos t-1,Entradas t-3,Entradas t-2,Entradas t-1,Salidas t-3,Salidas t-2,Salidas t-1,Sintomaticos t-3,Sintomaticos t-2,Sintomaticos t-1
Fecha,,,,,,,,,,,,,,,,,,,,,
2020-03-08,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-09,3.0,NaN,NaN,130.0,NaN,NaN,0.0,NaN,NaN,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2020-03-10,6.0,NaN,130.0,21.0,NaN,0.0,0.0,NaN,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0
2020-03-11,12.0,130.0,21.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,4.0
2020-03-12,9.0,21.0,39.0,116.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,10.0
2020-03-13,12.0,39.0,116.0,136.0,0.0,0.0,4.0,0.0,0.0,1.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,10.0,7.0
2020-03-14,23.0,116.0,136.0,228.0,0.0,4.0,8.0,0.0,1.0,1.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,9.0
2020-03-15,18.0,136.0,228.0,196.0,4.0,8.0,8.0,1.0,1.0,2.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,9.0,19.0
2020-03-16,24.0,228.0,196.0,121.0,8.0,8.0,13.0,1.0,2.0,2.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,19.0,14.0


Eliminamos los missing values (sujeto a posible reemplazamiento)

In [28]:
df_predictor.dropna(axis = 0, inplace = True)
df_predictor.head(3)

,Incidentes,Pruebas t-3,Pruebas t-2,Pruebas t-1,H t-3,H t-2,H t-1,H_UCI t-3,H_UCI t-2,H_UCI t-1,...,Asintomaticos t-1,Entradas t-3,Entradas t-2,Entradas t-1,Salidas t-3,Salidas t-2,Salidas t-1,Sintomaticos t-3,Sintomaticos t-2,Sintomaticos t-1
Fecha,,,,,,,,,,,,,,,,,,,,,
2020-04-02,71.0,250.0,616.0,598.0,283.0,304.0,287.0,59.0,57.0,58.0,...,13.0,59110.0,54693.0,55566.0,60070.0,56468.0,56640.0,31.0,20.0,23.0
2020-04-03,38.0,616.0,598.0,572.0,304.0,287.0,284.0,57.0,58.0,57.0,...,26.0,54693.0,55566.0,55754.0,56468.0,56640.0,56821.0,20.0,23.0,34.0
2020-04-04,43.0,598.0,572.0,460.0,287.0,284.0,284.0,58.0,57.0,57.0,...,6.0,55566.0,55754.0,59281.0,56640.0,56821.0,60228.0,23.0,34.0,37.0
